In [1]:
#Importing necessary libraries
from sklearn.datasets import load_boston
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
%matplotlib inline
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.feature_selection import RFE
from sklearn.linear_model import RidgeCV, LassoCV, Ridge, Lasso
from sklearn.model_selection import train_test_split
from sklearn import preprocessing

In [2]:
#Load in the data set
fifaDB = pd.read_csv("data.csv")

In [3]:
#Preview Data base
fifaDB.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18207 entries, 0 to 18206
Data columns (total 89 columns):
Unnamed: 0                  18207 non-null int64
ID                          18207 non-null int64
Name                        18207 non-null object
Age                         18207 non-null int64
Photo                       18207 non-null object
Nationality                 18207 non-null object
Flag                        18207 non-null object
Overall                     18207 non-null int64
Potential                   18207 non-null int64
Club                        17966 non-null object
Club Logo                   18207 non-null object
Value                       18207 non-null object
Wage                        18207 non-null object
Special                     18207 non-null int64
Preferred Foot              18159 non-null object
International Reputation    18159 non-null float64
Weak Foot                   18159 non-null float64
Skill Moves                 18159 non-null fl

In [4]:
fifaDB.columns

Index(['Unnamed: 0', 'ID', 'Name', 'Age', 'Photo', 'Nationality', 'Flag',
       'Overall', 'Potential', 'Club', 'Club Logo', 'Value', 'Wage', 'Special',
       'Preferred Foot', 'International Reputation', 'Weak Foot',
       'Skill Moves', 'Work Rate', 'Body Type', 'Real Face', 'Position',
       'Jersey Number', 'Joined', 'Loaned From', 'Contract Valid Until',
       'Height', 'Weight', 'LS', 'ST', 'RS', 'LW', 'LF', 'CF', 'RF', 'RW',
       'LAM', 'CAM', 'RAM', 'LM', 'LCM', 'CM', 'RCM', 'RM', 'LWB', 'LDM',
       'CDM', 'RDM', 'RWB', 'LB', 'LCB', 'CB', 'RCB', 'RB', 'Crossing',
       'Finishing', 'HeadingAccuracy', 'ShortPassing', 'Volleys', 'Dribbling',
       'Curve', 'FKAccuracy', 'LongPassing', 'BallControl', 'Acceleration',
       'SprintSpeed', 'Agility', 'Reactions', 'Balance', 'ShotPower',
       'Jumping', 'Stamina', 'Strength', 'LongShots', 'Aggression',
       'Interceptions', 'Positioning', 'Vision', 'Penalties', 'Composure',
       'Marking', 'StandingTackle', 'SlidingT

In [5]:
#Drop unnecessary columns
Fifa = fifaDB.drop(['LS', 'ST', 'RS', 'LW', 'LF', 'CF', 'RF', 'RW',
       'LAM', 'CAM', 'RAM', 'LM', 'LCM', 'CM', 'RCM', 'RM', 'LWB', 'LDM',
       'CDM', 'RDM', 'RWB', 'LB', 'LCB', 'CB', 'RCB', 'RB', 'Unnamed: 0', 'Photo', 'Flag', 'Club Logo', 'Joined', 'Loaned From', 'Contract Valid Until', 'Special', 'International Reputation', 'Real Face', "Nationality", "Age", "ID", "Release Clause", "Club","GKDiving", "GKHandling", "GKKicking", "GKPositioning", "GKReflexes" ], axis=1)

In [6]:
#Look for null values
Fifa.isna().sum() 

Name                0
Overall             0
Potential           0
Value               0
Wage                0
Preferred Foot     48
Weak Foot          48
Skill Moves        48
Work Rate          48
Body Type          48
Position           60
Jersey Number      60
Height             48
Weight             48
Crossing           48
Finishing          48
HeadingAccuracy    48
ShortPassing       48
Volleys            48
Dribbling          48
Curve              48
FKAccuracy         48
LongPassing        48
BallControl        48
Acceleration       48
SprintSpeed        48
Agility            48
Reactions          48
Balance            48
ShotPower          48
Jumping            48
Stamina            48
Strength           48
LongShots          48
Aggression         48
Interceptions      48
Positioning        48
Vision             48
Penalties          48
Composure          48
Marking            48
StandingTackle     48
SlidingTackle      48
dtype: int64

In [7]:
#Drop null values
Fifa = Fifa.dropna()

In [8]:
list_wages = Fifa['Wage']
wages = []
for i in list_wages:
    hoi = i.strip('K').strip('€')
    wage = int(hoi) * 1000
    wages.append(wage)
    
Fifa['Wage'] = wages



In [9]:
list_lengths = np.array(Fifa['Height'])
empty_list = []
for x in list_lengths:
    splitted_values = str(x).split("'")
    feet = (float(x[0])) + (float(x[2])/10)
    centimeters = feet * 30.48
    empty_list.append(round(centimeters))

Fifa['Height'] = empty_list


In [10]:
list_value = Fifa['Value']
list_new_values = []
rows = list_value[:]

for v in rows:
    if v[-1] == 'M':
        strip_value = v.strip('M€')
        if len(strip_value) >= 4:
            yes = (strip_value[:3])
            new_value = float(yes) * 1000000
            list_new_values.append(int(new_value))
        else:
            new_value = float(strip_value) * 1000000
            list_new_values.append(int(new_value))
            
    else:
        strip_value = v.strip('K€')
        new_value = float(strip_value) * 1000
        
        list_new_values.append(int(new_value))

        
Fifa['Value'] = list_new_values

In [11]:
list_weight = Fifa['Weight']
weights = []
for i in list_weight:
    lbs_to_kg = i.strip('lbs')
    convert_weight = float(lbs_to_kg)/2.2046
    roundup_weight = round(convert_weight)
    weights.append(roundup_weight)

Fifa['Weight'] = weights

In [12]:
#Label encoder for work rate
le = preprocessing.LabelEncoder()
Fifa['Work Rate'] = Fifa[['Work Rate']].apply(le.fit_transform)
Fifa['Position'] = Fifa[['Position']].apply(le.fit_transform)
Fifa['Body Type'] = Fifa[['Body Type']].apply(le.fit_transform)

In [13]:
#onehot encoding
Fifa["Preferred Foot"] = pd.get_dummies(Fifa["Preferred Foot"])

In [14]:
#Remove Goalkeepers
Fifa = Fifa[Fifa.Position != 5]
Fifa.head(10)

,Name,Overall,Potential,Value,Wage,Preferred Foot,Weak Foot,Skill Moves,Work Rate,Body Type,...,LongShots,Aggression,Interceptions,Positioning,Vision,Penalties,Composure,Marking,StandingTackle,SlidingTackle
0,L. Messi,94,94,110000000,565000,1,4.0,4.0,8,4,...,94.0,48.0,22.0,94.0,94.0,75.0,96.0,33.0,28.0,26.0
1,Cristiano Ronaldo,94,94,77000000,405000,0,4.0,5.0,1,1,...,93.0,63.0,29.0,95.0,82.0,85.0,95.0,28.0,31.0,23.0
2,Neymar Jr,92,93,118000000,290000,0,5.0,5.0,2,5,...,82.0,56.0,36.0,89.0,87.0,81.0,94.0,27.0,24.0,33.0
4,K. De Bruyne,91,92,102000000,355000,0,5.0,4.0,0,6,...,91.0,76.0,61.0,87.0,94.0,79.0,88.0,68.0,58.0,51.0
5,E. Hazard,91,91,93000000,340000,0,4.0,4.0,2,6,...,80.0,54.0,41.0,87.0,89.0,86.0,91.0,34.0,27.0,22.0
6,L. Modrić,91,91,67000000,420000,0,4.0,4.0,0,3,...,82.0,62.0,83.0,79.0,92.0,82.0,84.0,60.0,76.0,73.0
7,L. Suárez,91,91,80000000,455000,0,4.0,3.0,2,6,...,85.0,87.0,41.0,92.0,84.0,85.0,85.0,62.0,45.0,38.0
8,Sergio Ramos,91,91,51000000,380000,0,3.0,3.0,2,6,...,59.0,88.0,90.0,60.0,63.0,75.0,82.0,87.0,92.0,91.0
10,R. Lewandowski,90,90,77000000,205000,0,4.0,4.0,2,6,...,84.0,80.0,39.0,91.0,77.0,88.0,86.0,34.0,42.0,19.0
11,T. Kroos,90,90,76000000,355000,0,5.0,3.0,8,6,...,92.0,60.0,82.0,79.0,86.0,73.0,85.0,72.0,79.0,69.0


In [15]:
def savetocsv(dataframe, name):
    dataframe.to_csv(name, index = None, header=True)

In [16]:
savetocsv(Fifa, "NewFifa.csv")